In [ ]:
!pip install --user pykafka

In [ ]:
from pykafka import KafkaClient
client = KafkaClient(hosts="188.185.79.229:9092")
topic = client.topics['email_alert']

In [ ]:
consumer = topic.get_simple_consumer()
for message in consumer:
    print(message.value)

In [ ]:
!wget http://central.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.3.1/spark-streaming-kafka-0-8-assembly_2.11-2.3.1.jar

In [1]:
from pyspark.streaming.kafka import KafkaUtils
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.streaming.kafka import TopicAndPartition
import json

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
def sendingAlert(subject,count):
    return [subject,count,count>10]
spark.udf.register("sendingAlert", sendingAlert)

<function __main__.sendingAlert(subject, count)>

In [5]:
ssc = StreamingContext(sc, 5) #change frequency of getting data
ssc.checkpoint("_checkpoint")

In [6]:
kafkaStream =KafkaUtils.createDirectStream(ssc, ["email_alert"],
                                           {
                                               "metadata.broker.list":"188.185.79.229:9092",
                                               "auto.offset.reset":"smallest"
                                           }
                                          )

In [7]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [42]:
emails_dstream = parsed.map(lambda email: [email['subject'],email['timestamp'],1])\
                        .reduceByKeyAndWindow(lambda x,y: x+y, 10, 5)
# emails_dstream.pprint()
# benchmark=emails_dstream.map(lambda x:[x[0],x[1],x[1]>30])
benchmark=emails_dstream.map(lambda x: sendingAlert(x[0],x[1]))
# structuredOutput.foreachRDD(rdd => 
#   import sparkSession.implicits._
#   val df = rdd.toDF()
#   df.write.format("parquet").mode("append").save(s"$workDir/$targetFile")
# })
benchmark.pprint()
ssc.start()
ssc.awaitTerminationOrTimeout(10)


-------------------------------------------
Time: 2019-06-06 17:19:20
-------------------------------------------
['Cron <root@vocms041> cd /opt/CMSDataPopularity/xrootd; python2.6 lib/checkMViews.py', 32, True]
['PopDB cronJob of cron_2019-04-05_04-30-06.log', 1, False]
['WARNING: too many xrootd-cms-eos-popularity messages stored on the dashb brokers', 7, False]
['OK: xrootd-cms-eos-popularity messages stored on the dashb brokers', 5, False]
['PopDB cronJob of cron_2019-04-10_04-30-04.log', 1, False]
['The collector xrootd_2 in vocms043.cern.ch was not running', 4, False]
['Notification: Stop simplevisor at vocms043.cern.ch: 70%>=70%\r\n inodes', 12, True]
['Notification: Stop two insertion agents at vocms042.cern.ch', 2, False]
['PopDB cronJob of cron_2019-04-16_04-30-05.log', 1, False]
['PopDB cronJob of cron_2019-04-23_04-30-03.log', 1, False]
...



False

-------------------------------------------
Time: 2019-06-06 17:19:25
-------------------------------------------

-------------------------------------------
Time: 2019-06-06 17:19:30
-------------------------------------------

-------------------------------------------
Time: 2019-06-06 17:19:35
-------------------------------------------



In [56]:
ssc.stop()

<function __main__.sendingAlert(subject, count)>

In [ ]:
parsed.map().reduceByKeyAndWindow(,, windowDuration=)

In [ ]:
# subject_counts_sorted_dstream = subject_counts.transform(\
#   (lambda foo:foo\
#    .sortBy(lambda x:( -x[1]))))
# top_five_subjects = subject_counts_sorted_dstream.transform\
#   (lambda rdd:sc.parallelize(rdd.take(5)))
# top_five_subjects.pprint()

In [ ]:
ssc.stop()

In [ ]:
!wget http://central.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8_2.11/2.0.0-preview/spark-streaming-kafka-0-8_2.11-2.0.0-preview.jar